# Question 1
Resolve the position-level forward and inverse kinematics of the Furuta pendulum with the help of the coordinate systems shown as shown in the figure. Once you have the kinematic equations, answer the following questions:

(a) If the joint angles $ \theta $ are given by 

$$
\theta = \begin{bmatrix} \theta_1 \\ \theta_2 \end{bmatrix} = \begin{bmatrix} \pi/3 \\ -3\pi/7 \end{bmatrix}
$$

what is the pose of the end-effector, i.e., the frame $\{C\}$ with respect to the fixed frame $\{0\}$? Provide the orientation using both the rotation matrix and the corresponding quaternion.

(b) At the same angles given in part (a), what is the pose of frame $\{2\}$ with respect to the fixed frame $\{0\}$? Provide the orientation using both the rotation matrix and the corresponding quaternion.

(c) Provide the axis-angle representation of the orientation for parts (a) and (b). Use rodrigues’s formula to recover the rotation matrices you found in parts (a) and (b).

(d) Suppose a camera is installed at the end-effector of the Furuta pendulum. Let $\{C\}$ denote the camera frame, as shown. What is the value of $ \theta_1 $ and $ \theta_2 $ if the $ z $-axis of the camera points in the direction $ k $, where

$$
k = -\frac{1}{2} x_0 + \frac{1}{2} y_0 - \frac{\sqrt{2}}{2} z_0.
$$

(e) Repeat part (d) to find the value of $ \theta_1 $ and $ \theta_2 $ if the origin of frame $\{C\}$ is located at a point $ P $, whose coordinate vector is given in frame $\{0\}$ by

$$
p = 1.075x_0 + 0.303y_0 + 1.022z_0.
$$


In [434]:
import sympy as sp
import numpy as np
from spatialmath import SO3, SE3
from spatialmath.base import r2q, qprint, tr2angvec, skew, vex
from scipy.linalg import expm # used only for the matrix exponential part of rodrigues formula

## Solve Forward Kinimatics
Resolve the position-level forward kinematics of the Furuta pendulum with the help of the coordinate systems shown as shown in the figure.

Find:<br>
$^{0}T_{c} = {}^{0}T_{1} \, {}^{1}T_{2} \, {}^{2}T_{c}$

In [435]:
d1, d2, d3 = sp.symbols('d1 d2 d3')

# Find the position matrixes
t01 = sp.Matrix([0,0,d1]) #read as the position of frame 1 in respect to frame 0
t12 = sp.Matrix([0,0,d2])
t2c = sp.Matrix([0,0,d3])

In [436]:
theta1, theta2 = sp.symbols('theta1 theta2')
# - Find the Rotation Matrixes
# Find R01 (read as the rotation matrix of 1 in respect to 0)

Rx90 = SO3.Rx(sp.pi/2) #90 degrees
RyTheta1 = SO3.Ry(theta1)
R01 = Rx90 * RyTheta1

sp.Matrix(R01.A)

Matrix([
[cos(theta1), 0,  sin(theta1)],
[sin(theta1), 0, -cos(theta1)],
[          0, 1,            0]])

In [437]:
# Find R12

RyTheta2 = SO3.Ry(theta2)
R12 = Rx90 * RyTheta2

sp.Matrix(R12.A)

Matrix([
[cos(theta2), 0,  sin(theta2)],
[sin(theta2), 0, -cos(theta2)],
[          0, 1,            0]])

In [438]:
# Find R2c

Rz270 = SO3.Rz(-sp.pi/2)
R2c = Rz270

sp.Matrix(R2c.A)

Matrix([
[ 0, 1, 0],
[-1, 0, 0],
[ 0, 0, 1]])

Using the position and rotation matrixes, construct coorponding pose.

In [439]:
def make_pose(R, t): # this function is needed since we are using sympy
    """
    Creates a 4x4 homogeneous transformation matrix T from a rotation matrix R and a translation vector t.
    
    Parameters:
      R : A rotation matrix. This can be an object with a .A attribute (like an SO3 object)
          or a standard Sympy Matrix.
      t : A 3x1 position vector. It should be a column vector.
      
    Returns:
      T : A 4x4 homogeneous transformation matrix as a Sympy Matrix.
    """
    # Convert the rotation matrix to a standard Sympy Matrix if necessary.
    try:
        R_mat = sp.Matrix(R.A)
    except AttributeError:
        R_mat = sp.Matrix(R)
    
    # Ensure t is a column vector.
    t_vec = sp.Matrix(t)
    if t_vec.shape[1] != 1:
        t_vec = t_vec.T if t_vec.shape[0] == 1 else t_vec
    
    # Build the transformation matrix by concatenating R and t, and appending the bottom row.
    T = sp.Matrix.vstack(
        sp.Matrix.hstack(R_mat, t_vec),
        sp.Matrix([[0, 0, 0, 1]])
    )
    return T

In [440]:
T01 = make_pose(R01, t01)
T01

Matrix([
[cos(theta1), 0,  sin(theta1),  0],
[sin(theta1), 0, -cos(theta1),  0],
[          0, 1,            0, d1],
[          0, 0,            0,  1]])

In [441]:
T12 = make_pose(R12, t12)
T12

Matrix([
[cos(theta2), 0,  sin(theta2),  0],
[sin(theta2), 0, -cos(theta2),  0],
[          0, 1,            0, d2],
[          0, 0,            0,  1]])

In [442]:
T2c = make_pose(R2c, t2c)
T2c

Matrix([
[ 0, 1, 0,  0],
[-1, 0, 0,  0],
[ 0, 0, 1, d3],
[ 0, 0, 0,  1]])

In [443]:
# Construct final pose

T0c = T01 * T12 * T2c
T0c

Matrix([
[-sin(theta1), cos(theta1)*cos(theta2), sin(theta2)*cos(theta1),  d2*sin(theta1) + d3*sin(theta2)*cos(theta1)],
[ cos(theta1), sin(theta1)*cos(theta2), sin(theta1)*sin(theta2), -d2*cos(theta1) + d3*sin(theta1)*sin(theta2)],
[           0,             sin(theta2),            -cos(theta2),                          d1 - d3*cos(theta2)],
[           0,                       0,                       0,                                            1]])

In [444]:
# extract final rotation matrix
R0c = T0c[:3,:3]
R0c

Matrix([
[-sin(theta1), cos(theta1)*cos(theta2), sin(theta2)*cos(theta1)],
[ cos(theta1), sin(theta1)*cos(theta2), sin(theta1)*sin(theta2)],
[           0,             sin(theta2),            -cos(theta2)]])

In [445]:
# extract final position matrix
t0c = T0c[:3, 3]
t0c

Matrix([
[ d2*sin(theta1) + d3*sin(theta2)*cos(theta1)],
[-d2*cos(theta1) + d3*sin(theta1)*sin(theta2)],
[                         d1 - d3*cos(theta2)]])

## Solve Inverse Kinematics 

Resolve the position-level *inverse kinematics* of the Furuta pendulum with the help of the coordinate systems shown as shown in the figure. 

Determine $ \theta_1 $ and $ \theta_2 $ required to achieve the desired position $ t $.

### Position Equations:
The desired position is given by:

$ {}^0t_c = [{}^0x_c, {}^0y_c, {}^0z_c] $

where:

$ {}^0x_c = f(\theta_1, \theta_2) $

$ {}^0y_c = f(\theta_1, \theta_2) $

$ {}^0z_c = f(\theta_1, \theta_2) $

From these equations, $ \theta_1 $ and $ \theta_2 $ can be solved to achieve the desired position.


In [446]:
xc = sp.symbols(r'{}^0x_c')
yc = sp.symbols(r'{}^0y_c')
zc = sp.symbols(r'{}^0z_c')

# Relate t0c with xc, yc, and zc
eq1 = sp.Eq(xc, t0c[0]) #toc[0] extracts the x component of the position of frame c in repect to frame 0
eq1

Eq({}^0x_c, d2*sin(theta1) + d3*sin(theta2)*cos(theta1))

In [447]:
eq2 = sp.Eq(yc, t0c[1])
eq2

Eq({}^0y_c, -d2*cos(theta1) + d3*sin(theta1)*sin(theta2))

In [448]:
eq3 = sp.Eq(zc, t0c[2])
eq3

Eq({}^0z_c, d1 - d3*cos(theta2))

In [449]:
# Now solve
solutions_for_theta = sp.solve([eq1, eq2, eq3], (theta1, theta2), check=False)
# List of 4 possible (theta1, theta2) solutions (see nicely formated version just below)
solutions_for_theta


[(-2*atan((d3*sqrt((-d1 + d3 + {}^0z_c)*(d1 + d3 - {}^0z_c)/d3**2) - sqrt(-d1**2 + 2*d1*{}^0z_c + d2**2 + d3**2 - {}^0y_c**2 - {}^0z_c**2))/(d2 - {}^0y_c)),
  acos((d1 - {}^0z_c)/d3)),
 (2*atan((d3*sqrt((-d1 + d3 + {}^0z_c)*(d1 + d3 - {}^0z_c)/d3**2) - sqrt(-d1**2 + 2*d1*{}^0z_c + d2**2 + d3**2 - {}^0y_c**2 - {}^0z_c**2))/(d2 - {}^0y_c)),
  -acos((d1 - {}^0z_c)/d3) + 2*pi),
 (-2*atan((d3*sqrt((-d1 + d3 + {}^0z_c)*(d1 + d3 - {}^0z_c)/d3**2) + sqrt(-d1**2 + 2*d1*{}^0z_c + d2**2 + d3**2 - {}^0y_c**2 - {}^0z_c**2))/(d2 - {}^0y_c)),
  acos((d1 - {}^0z_c)/d3)),
 (2*atan((d3*sqrt((-d1 + d3 + {}^0z_c)*(d1 + d3 - {}^0z_c)/d3**2) + sqrt(-d1**2 + 2*d1*{}^0z_c + d2**2 + d3**2 - {}^0y_c**2 - {}^0z_c**2))/(d2 - {}^0y_c)),
  -acos((d1 - {}^0z_c)/d3) + 2*pi)]

In [450]:
# # Prints nicely formated Markdown and LaTaX format of all the solutions for copy and pasting
# sol_num = 1
# print("### Inverse Kinematics Solutions")
# print("___")
# for a in solutions:
#     print()
#     print("Solution %d:\n" % sol_num)
#     print("$", end=" \\theta_1 = ")
#     print(sp.latex(a[0]), end=" $")
#     print()
#     print()
#     print("$", end=" \\theta_2 = ")
#     print(sp.latex(a[1]), end=" $")
#     print()
#     print("___")
#     sol_num += 1

### Inverse Kinematics Solutions
___

Solution 1:

$ \theta_1 = - 2 \operatorname{atan}{\left(\frac{d_{3} \sqrt{\frac{\left(- d_{1} + d_{3} + {}^0z_c\right) \left(d_{1} + d_{3} - {}^0z_c\right)}{d_{3}^{2}}} - \sqrt{- d_{1}^{2} + 2 d_{1} {}^0z_c + d_{2}^{2} + d_{3}^{2} - \left({}^0y_c\right)^{2} - \left({}^0z_c\right)^{2}}}{d_{2} - {}^0y_c} \right)} $

$ \theta_2 = \operatorname{acos}{\left(\frac{d_{1} - {}^0z_c}{d_{3}} \right)} $
___

Solution 2:

$ \theta_1 = 2 \operatorname{atan}{\left(\frac{d_{3} \sqrt{\frac{\left(- d_{1} + d_{3} + {}^0z_c\right) \left(d_{1} + d_{3} - {}^0z_c\right)}{d_{3}^{2}}} - \sqrt{- d_{1}^{2} + 2 d_{1} {}^0z_c + d_{2}^{2} + d_{3}^{2} - \left({}^0y_c\right)^{2} - \left({}^0z_c\right)^{2}}}{d_{2} - {}^0y_c} \right)} $

$ \theta_2 = - \operatorname{acos}{\left(\frac{d_{1} - {}^0z_c}{d_{3}} \right)} + 2 \pi $
___

Solution 3:

$ \theta_1 = - 2 \operatorname{atan}{\left(\frac{d_{3} \sqrt{\frac{\left(- d_{1} + d_{3} + {}^0z_c\right) \left(d_{1} + d_{3} - {}^0z_c\right)}{d_{3}^{2}}} + \sqrt{- d_{1}^{2} + 2 d_{1} {}^0z_c + d_{2}^{2} + d_{3}^{2} - \left({}^0y_c\right)^{2} - \left({}^0z_c\right)^{2}}}{d_{2} - {}^0y_c} \right)} $

$ \theta_2 = \operatorname{acos}{\left(\frac{d_{1} - {}^0z_c}{d_{3}} \right)} $
___

Solution 4:
...
$ \theta_1 = 2 \operatorname{atan}{\left(\frac{d_{3} \sqrt{\frac{\left(- d_{1} + d_{3} + {}^0z_c\right) \left(d_{1} + d_{3} - {}^0z_c\right)}{d_{3}^{2}}} + \sqrt{- d_{1}^{2} + 2 d_{1} {}^0z_c + d_{2}^{2} + d_{3}^{2} - \left({}^0y_c\right)^{2} - \left({}^0z_c\right)^{2}}}{d_{2} - {}^0y_c} \right)} $

$ \theta_2 = - \operatorname{acos}{\left(\frac{d_{1} - {}^0z_c}{d_{3}} \right)} + 2 \pi $
___

## Part A

(a) If the joint angles $ \theta $ are given by 

$$
\theta = \begin{bmatrix} \theta_1 \\ \theta_2 \end{bmatrix} = \begin{bmatrix} \pi/3 \\ -3\pi/7 \end{bmatrix}
$$

what is the pose of the end-effector, i.e., the frame $\{C\}$ with respect to the fixed frame $\{0\}$,  ${}^0T_c$? Provide the orientation using both the rotation matrix and the corresponding quaternion.

In [451]:
# With the given angles, the pose is now:
T0c_parta = T0c.subs({d1: 1.2, d2: 0.8, d3: 0.8, theta1: sp.pi /3, theta2: -3 * sp.pi / 7})
print("The pose of frame c in repect to frame 0:")
T0c_parta

The pose of frame c in repect to frame 0:


Matrix([
[-sqrt(3)/2,         cos(3*pi/7)/2,         -sin(3*pi/7)/2, -0.4*sin(3*pi/7) + 0.4*sqrt(3)],
[       1/2, sqrt(3)*cos(3*pi/7)/2, -sqrt(3)*sin(3*pi/7)/2, -0.4*sqrt(3)*sin(3*pi/7) - 0.4],
[         0,          -sin(3*pi/7),           -cos(3*pi/7),          1.2 - 0.8*cos(3*pi/7)],
[         0,                     0,                      0,                              1]])

In [452]:
# Extract the rotation matrix
R0c_parta = T0c_parta[:3,:3]
print("The rotation matrix: ")

R0c_parta


The rotation matrix: 


Matrix([
[-sqrt(3)/2,         cos(3*pi/7)/2,         -sin(3*pi/7)/2],
[       1/2, sqrt(3)*cos(3*pi/7)/2, -sqrt(3)*sin(3*pi/7)/2],
[         0,          -sin(3*pi/7),           -cos(3*pi/7)]])

In [453]:
# Quaternian representation of the Rotation matrix
R0c_parta = np.array(R0c_parta, dtype=float) #convert R to a np array
q_parta = r2q(R0c_parta)
print("The quaternian representation of the roatation:\n")

qprint(q_parta)


The quaternian representation of the roatation:

-0.1614 <  0.2024,  0.7552, -0.6022 >


## Part B
(b) At the same angles given in part (a), what is the pose of frame $\{2\}$ with respect to the fixed frame $\{0\}$? Provide the orientation using both the rotation matrix and the corresponding quaternion.


In [454]:
# the pose is
T02 = T01 * T12
T02_partb = T02.subs({d1: 1.2, d2: 0.8, d3: 0.8, theta1: sp.pi /3, theta2: -3 * sp.pi / 7})
print("The pose of frame 2 in respect to frame 0:")
T02_partb

The pose of frame 2 in respect to frame 0:


Matrix([
[        cos(3*pi/7)/2, sqrt(3)/2,         -sin(3*pi/7)/2, 0.4*sqrt(3)],
[sqrt(3)*cos(3*pi/7)/2,      -1/2, -sqrt(3)*sin(3*pi/7)/2,        -0.4],
[         -sin(3*pi/7),         0,           -cos(3*pi/7),         1.2],
[                    0,         0,                      0,           1]])

In [455]:
# so the rotation matrix is
R02_partb = T02_partb[:3,:3]
print("The rotation matrix:")

R02_partb


The rotation matrix:


Matrix([
[        cos(3*pi/7)/2, sqrt(3)/2,         -sin(3*pi/7)/2],
[sqrt(3)*cos(3*pi/7)/2,      -1/2, -sqrt(3)*sin(3*pi/7)/2],
[         -sin(3*pi/7),         0,           -cos(3*pi/7)]])

In [456]:
#and the quaternian representation of the rotation matrix is
R02_partb = np.array(R02_partb, dtype=float) #convert R to a np array
q_partb = r2q(R02_partb)
print("The quaternian representation of the roatation:\n")
qprint(q_partb)


The quaternian representation of the roatation:

 0.3117 <  0.6771,  0.3909, -0.5400 >


## Part C
(c) Provide the axis-angle representation of the orientation for parts (a) and (b). Use Rodrigues’s formula to recover the rotation matrices you found in parts (a) and (b).


In [457]:
ang0c, axis0c = tr2angvec(R0c_parta, unit='rad')

print("The angle axis representation of frame c in respect to frame 0:\n")
print("\tAngle: %.3f rad" % (ang0c))
print("\t    or %.3f degrees" % np.rad2deg(ang0c))
print("\tVector: " + str(axis0c))

The angle axis representation of frame c in respect to frame 0:

	Angle: 2.817 rad
	    or 161.427 degrees
	Vector: [-0.20504017 -0.76522035  0.61024286]


In [458]:
ang02, axis02 = tr2angvec(R02_partb, unit='rad')

print("The angle axis representation of frame 2 in respect to frame 0:\n")
print("\tAngle: %.3f rad" % (ang02))
print("\t    or %.3f degrees" % np.rad2deg(ang02))

print("\tVector: " + str(axis02))

The angle axis representation of frame 2 in respect to frame 0:

	Angle: 2.508 rad
	    or 143.671 degrees
	Vector: [ 0.71259763  0.41141843 -0.56827765]


The equation represents the matrix exponential of a skew-symmetric matrix, which is used in 3D rotation transformations based on the axis-angle representation.

$ R = e^{\widehat{\omega} \theta} = \mathbf{I} + \sin(\theta) \widehat{\omega} + (1 - \cos(\theta)) \widehat{\omega}^2 $

Where:
- $\omega$ is the **axis of rotation**, a unit vector that defines the direction of rotation.
- $\theta$ is the **scalar part of the axis-angle rotation**, representing the rotation magnitude in radians.
- $\widehat{\omega}$ is the **skew-symmetric matrix** of $\omega$, used to compute the rotation matrix.
- $\mathbf{I}$ is the **identity matrix**.


In [459]:
#this solves it with matrix exponential for C in 0
R0c_parta_matrixExponential = expm(skew(axis0c) * ang0c)

print("Recovered rotation matrix using matrix exponential: \n(frame c in respect to frame 0) ")
sp.Matrix(R0c_parta_matrixExponential.round(3)) # sympy for nice printing

Recovered rotation matrix using matrix exponential: 
(frame c in respect to frame 0) 


Matrix([
[-0.866,  0.111, -0.487],
[   0.5,  0.193, -0.844],
[   0.0, -0.975, -0.223]])

In [460]:
# this solves it using rodrigues formula for C in 0
R0c_parta_rodrigues = SO3() + np.sin(ang0c) * skew(axis0c) + (1 - np.cos(ang0c)) * (np.dot(skew(axis0c), skew(axis0c)))

print("Recovered rotation matrix using Rodrigues' formula: \n(frame c in respect to frame 0) ")
sp.Matrix(R0c_parta_rodrigues.round(3)) # sympy for nice printing

Recovered rotation matrix using Rodrigues' formula: 
(frame c in respect to frame 0) 


Matrix([
[-0.866,  0.111, -0.487],
[   0.5,  0.193, -0.844],
[   0.0, -0.975, -0.223]])

In [461]:
# This is our original R0c matrix for comparing
print("As can be seen, Rodrigues' formula correctly recovered the rotation matrix from the angle axis representation.\n")

print("The original rotation matrix: \n(frame c in respect to frame 0) ")
sp.Matrix(R0c_parta.round(3))


As can be seen, Rodrigues' formula correctly recovered the rotation matrix from the angle axis representation.

The original rotation matrix: 
(frame c in respect to frame 0) 


Matrix([
[-0.866,  0.111, -0.487],
[   0.5,  0.193, -0.844],
[   0.0, -0.975, -0.223]])

In [462]:
#this solves it with matrix exponential for 2 in 0
R02_parta_matrixExponential = expm(skew(axis02) * ang02)

print("Recovered rotation matrix using matrix exponential: \n(frame 2 in respect to frame 0) ")
sp.Matrix(R02_parta_matrixExponential.round(3)) # sympy for nice printing

Recovered rotation matrix using matrix exponential: 
(frame 2 in respect to frame 0) 


Matrix([
[ 0.111, 0.866, -0.487],
[ 0.193,  -0.5, -0.844],
[-0.975,   0.0, -0.223]])

In [463]:
# this solves it using rodrigues formula for 2 in 0
R02_parta_rodrigues = SO3() + np.sin(ang02) * skew(axis02) + (1 - np.cos(ang02)) * (np.dot(skew(axis02), skew(axis02)))

print("Recovered rotation matrix using Rodrigues' formula: \n(frame 2 in respect to frame 0) ")
sp.Matrix(R02_parta_rodrigues.round(3)) # sympy for nice printing

Recovered rotation matrix using Rodrigues' formula: 
(frame 2 in respect to frame 0) 


Matrix([
[ 0.111, 0.866, -0.487],
[ 0.193,  -0.5, -0.844],
[-0.975,   0.0, -0.223]])

In [464]:
# This is our original R0c matrix for comparing
print("As can be seen, Rodrigues' formula correctly recovered the rotation matrix from the angle axis representation.\n")

print("The original rotation matrix: \n(frame 2 in respect to frame 0) ")
sp.Matrix(R02_partb.round(3))


As can be seen, Rodrigues' formula correctly recovered the rotation matrix from the angle axis representation.

The original rotation matrix: 
(frame 2 in respect to frame 0) 


Matrix([
[ 0.111, 0.866, -0.487],
[ 0.193,  -0.5, -0.844],
[-0.975,   0.0, -0.223]])

## Part D

(d) Suppose a camera is installed at the end-effector of the Furuta pendulum. Let $\{C\}$ denote the camera frame, as shown. What is the value of $ \theta_1 $ and $ \theta_2 $ if the $ z $-axis of the camera points in the direction $ k $, where

$$
k = -\frac{1}{2} x_0 + \frac{1}{2} y_0 - \frac{\sqrt{2}}{2} z_0.
$$

We need to find the solutions for $\theta_1$ and $\theta_2$ in this equation:

$$
\begin{bmatrix} 
\ {}^0z_{c,1} \\ 
\ {}^0z_{c,2} \\ 
\ {}^0z_{c,3} 
\end{bmatrix} 
=
\begin{bmatrix} 
k_1 \\ 
k_2 \\ 
k_3 
\end{bmatrix}


\\
or
\\


\begin{bmatrix} 
\sin{\left(\theta_{2} \right)} \cos{\left(\theta_{1} \right)} \\ 
\sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2} \right)} \\ 
- \cos{\left(\theta_{2} \right)} 
\end{bmatrix} 
=
\begin{bmatrix} 
-0.5 \\ 
0.5 \\ 
- \frac{\sqrt{2}}{2} 
\end{bmatrix}
$$


In [465]:
# Our understanding of this question is that k is the same thing as the z axis part of R0c
# or in other words
print("k is the same thing as the z part of the rotation matrix: ")
R0c[:,2] # the z part of the rotation matrix

k is the same thing as the z part of the rotation matrix: 


Matrix([
[sin(theta2)*cos(theta1)],
[sin(theta1)*sin(theta2)],
[           -cos(theta2)]])

In [466]:
# define k
k = sp.Matrix([-1/2, 1/2, -sp.sqrt(2)/2])
# set up equations
eq1_partd = sp.Eq(R0c[0,2], k[0])
eq2_partd = sp.Eq(R0c[1,2], k[1])
eq3_partd = sp.Eq(R0c[2,2], k[2])

# solve for theta1 and theta2
solutionPartD = sp.solve([eq1_partd, eq2_partd, eq3_partd], (theta1, theta2), check=False)

#The 4 different solutions for theta1 and theta2 in rad
print("The solutions for theta1 and theta2 to satisfy k are given here in radians.")
print("Each row is one of four possible solutions. Columns 1 and 2 represet theta1 and theta2 respectivly.")
solutionPartD

The solutions for theta1 and theta2 to satisfy k are given here in radians.
Each row is one of four possible solutions. Columns 1 and 2 represet theta1 and theta2 respectivly.


[(-0.785398163397448, 5.49778714378214),
 (0.785398163397448, 0.785398163397448),
 (2.35619449019234, 0.785398163397448),
 (3.92699081698724, 5.49778714378214)]

In [467]:
#convert theta1 and theta2 into degrees
solutionPartD_list = []
for a in solutionPartD:
    solutionPartD_list.append(a)
solutionPartD_deg = np.rad2deg(np.array(solutionPartD_list, dtype=float))

print("The same solutions from above in degrees:")
print("Each row is one of four possible solutions. Columns 1 and 2 represet theta1 and theta2 respectivly.")
sp.Matrix(solutionPartD_deg)

The same solutions from above in degrees:
Each row is one of four possible solutions. Columns 1 and 2 represet theta1 and theta2 respectivly.


Matrix([
[-45.0, 315.0],
[ 45.0,  45.0],
[135.0,  45.0],
[225.0, 315.0]])

We are given 4 solutions here, lets check them all to see which ones make sense.

We will do this by plugging theta into the pose of frame c in respect to frame 0 and comparing it to the orientation of k to the orientation of the z axis. If they are the same, then we have correctly found angles to make the end effector have that orientation.

In [468]:
correct_solutions = []
corresponding_theta = []
# Plug in solution to pose, check to see if the z part of the rotation matrix matches up with k
for theta_iterate in solutionPartD:
    solution_on_trial = T0c.subs({theta1: theta_iterate[0], theta2: theta_iterate[1], d1: 1.2, d2: 0.8, d3: 0.8})
    dif = solution_on_trial[:3,2] - k # solution_on_trial is the pose, index [:3,2] extracts z part of the rotation matrix
    if dif[0] < 0.00001 and dif[1] < 0.00001 and dif[2] < 0.00001:
        correct_solutions.append(solution_on_trial) 
        corresponding_theta.append(theta_iterate)

print("\nCorrect solutions found:\n")
for a in range(len(correct_solutions)):
    print("Solution %d:" % (a+1))
    print("Theta1 and theta2:", end="\n\t")
    sp.pprint(corresponding_theta[a])
    print("Result in this pose:\n")
    sp.pprint(correct_solutions[a])
    print()



Correct solutions found:

Solution 1:
Theta1 and theta2:
	(-0.785398163397448, 5.49778714378214)
Result in this pose:

⎡0.707106781186547         0.5                 -0.5         -0.965685424949238⎤
⎢                                                                             ⎥
⎢0.707106781186548         -0.5                0.5          -0.165685424949238⎥
⎢                                                                             ⎥
⎢        0          -0.707106781186548  -0.707106781186547  0.634314575050762 ⎥
⎢                                                                             ⎥
⎣        0                  0                   0                   1         ⎦

Solution 2:
Theta1 and theta2:
	(2.35619449019234, 0.785398163397448)
Result in this pose:

⎡-0.707106781186548        -0.5                -0.5         0.165685424949238⎤
⎢                                                                            ⎥
⎢-0.707106781186547         0.5                0.5          0.9656854

We can see from the 4 solutions, only 2 correctly represent positions that match k. To narrow down only one solution, we would need position information.

# Part E
(e) Repeat part (d) to find the value of $ \theta_1 $ and $ \theta_2 $ if the origin of frame $\{C\}$ is located at a point $ P $, whose coordinate vector is given in frame $\{0\}$ by

$$
p = 1.075x_0 + 0.303y_0 + 1.022z_0.
$$


Remember from the Inverse Kinematics section that we found these functions:

$ \theta_1 = f({}^0x_c, {}^0y_c, {}^0z_c) $

$ \theta_2 = f({}^0x_c, {}^0y_c, {}^0z_c) $

In the following code, we will be plugging $ p $ into $ {}^0x_c, {}^0y_c, $ and $ {}^0z_c $ and figuring out what $ \theta_1 $ and $ \theta_2 $ could be. 

In [ ]:
theta_on_trial_dict = {
    d1: 1.2, 
    d2: 0.8, 
    d3: 0.8,
    xc: 1.075,
    yc: 0.303,
    zc: 1.022
}

# Plug p into solutions_for_theta to get possible outputs of theta
theta_options_partE = []
for a in range(len(solutions_for_theta)):
    temp_list = []
    for b in range(len(solutions_for_theta[0])):
        theta_current_value = solutions_for_theta[a][b].subs(theta_on_trial_dict)
        temp_list.append(theta_current_value)
    theta_options_partE.append(tuple(temp_list))

print("All possible solution pairs for theta that correspond to point p in radians.")
theta_options_partE

All possible solution pairs for theta that correspond to point p in radians.


[(1.07272370972679, 1.34641832379787),
 (-1.07272370972679, -1.34641832379787 + 2*pi),
 (-2.61813591673788, 1.34641832379787),
 (2.61813591673788, -1.34641832379787 + 2*pi)]

Now we need to check which solutions make sense. Lets plug in these theta values and see if we get the correct position after.

In [ ]:
p = sp.Matrix([1.075, 0.303, 1.022])

# Plug in theta to pose, check to see if the position of the pose matches up with p
correct_solutions = []
corresponding_theta = []
for theta_iterate in theta_options_partE:
    pose_on_trial = T0c.subs({theta1: theta_iterate[0], theta2: theta_iterate[1], d1: 1.2, d2: 0.8, d3: 0.8})
    dif = pose_on_trial[:3,3] - p # pose_on_trial is the pose, index [:3,3] extracts the position
    if dif[0] < 0.00001 and dif[1] < 0.00001 and dif[2] < 0.00001:
        correct_solutions.append(pose_on_trial) 
        corresponding_theta.append(theta_iterate)

print("\n-- Correct solutions found: --\n")
for a in range(len(correct_solutions)):
    print("Solution %d:" % (a+1))
    print("Theta1 and theta2 in rad:", end="\n\t")
    sp.pprint(corresponding_theta[a])
    print("Result in this pose:\n")
    sp.pprint(correct_solutions[a])
    print()



-- Correct solutions found: --

Solution 1:
Theta1 and theta2 in rad:
	(-1.07272370972679, -1.34641832379787 + 2⋅π)
Result in this pose:

⎡0.878504967894967  0.106295639561954   -0.465757724996567  -1.07541015431323⎤
⎢                                                                            ⎥
⎢0.477733211514401  -0.19546735535663   0.856483211514401         0.303      ⎥
⎢                                                                            ⎥
⎢        0          -0.974932689984288       -0.2225              1.022      ⎥
⎢                                                                            ⎥
⎣        0                  0                   0                   1        ⎦

Solution 2:
Theta1 and theta2 in rad:
	(-2.61813591673788, 1.34641832379787)
Result in this pose:

⎡0.499876985793974   -0.192706452208752  -0.84438570709756   -1.07541015431323⎤
⎢                                                                             ⎥
⎢-0.866096414421357  -0.111222629339159  -0.4873